## Basic agent with two tools: text retrieval and visual retrieval

In [1]:
from src.langchain.chains.movie_rag import MovieRAGChain
from src. langchain.loaders import MoviePosterDocumentLoader
from src.retrievers.visual_retriever import VisualRetriever
from src.langchain.prompts import ZERO_SHOT_QA_PROMPT

In [2]:
poster_loader = MoviePosterDocumentLoader('/Users/saghar/Desktop/movie-rag/datasets/rotten-tomatoes-reviews/prep/movie_posters.csv', max_movies=1000)   # small number since model is local
visual_docs = poster_loader.load()
visual_retriever = VisualRetriever(model_name="ViT-B/32", use_text_fusion=True, alpha=0.8)
visual_retriever.add_documents(visual_docs)

PLOTS_PATH = "/Users/saghar/Desktop/movie-rag/datasets/rotten-tomatoes-reviews/prep/movie_plots.csv"
REVIEWS_PATH = "/Users/saghar/Desktop/movie-rag/datasets/rotten-tomatoes-reviews/prep/reviews_w_movies_full.csv"
MAX_MOVIES = 500  # Limit for faster demos

full_chain = MovieRAGChain(
    plots_path=PLOTS_PATH,
    reviews_path=REVIEWS_PATH,
    max_movies=MAX_MOVIES,
    use_custom_retriever=True,
    use_custom_chunk=True,
    custom_prompt=ZERO_SHOT_QA_PROMPT,
    k=5,
    use_hyde=True,
    hyde_model="gpt-4o-mini",
    use_reranking=True,
    reranker_cfg={'type':'llm'},
    initial_k=20, 
)
full_chain.build()

Loading posters from /Users/saghar/Desktop/movie-rag/datasets/rotten-tomatoes-reviews/prep/movie_posters.csv...
Created 947 poster docs.
  ✓ 947 poster documents
Loading CLIP model: ViT-B/32
✓ CLIP loaded on cpu
✓ VisualRetriever (text_fusion=True, method=weight_average)
  Alpha: 0.8 (image=0.8, text=0.2)
Encoding 947 posters with CLIP...
Encoding 947 text descriptions with CLIP...
✓ Fused embeddings using weight_average method (dim=512)
✓ Added 947 posters to index
✓ MovieRAGChain initialized
  Retriever type: custom + reranking + HyDE
  LLM: gpt-4o-mini

Building RAG Pipeline

1. Loading documents...
Limiting to 500 movies
Loading plots from /Users/saghar/Desktop/movie-rag/datasets/rotten-tomatoes-reviews/prep/movie_plots.csv...
Created 383 plot docs.
  ✓ 383 plot documents
Loading reviews from /Users/saghar/Desktop/movie-rag/datasets/rotten-tomatoes-reviews/prep/reviews_w_movies_full.csv...
Created 500 review docs.
  ✓ 500 review documents
✓ Total: 883 reviews and plots documents

2

#### Creating and testing tools

In [3]:
from src.agents.tools import TextMovieTool, VisualMovieTool, CombinedMovieTool

In [13]:
# Create tools
text_tool = TextMovieTool(full_chain).get_tool()
visual_tool = VisualMovieTool(visual_retriever).get_tool()
combined_tool = CombinedMovieTool(full_chain, visual_retriever).get_tool()

# Test text tool
print("=" * 60)
print("Testing TEXT tool:")
print("=" * 60)
result = text_tool.invoke("What is a funny romantic movie set in Europe?")
print(result)

print("\n" + "=" * 60)
print("Testing VISUAL tool:")
print("=" * 60)
result = visual_tool.invoke("dark moody sci-fi films")
print(result)

print("\n" + "=" * 60)
print("Testing COMBINED tool:")
print("=" * 60)
result = combined_tool.invoke("what cartoon is about family and has bright visuals?")
print(result)


Testing TEXT tool:
A funny romantic movie set in Europe is "A Royal Night Out." It features a whimsical depiction of British princesses partying with commoners at the end of World War II, blending romance and comedy in a light-hearted manner. Another option is "A Good Year," which is set in southern France and involves themes of love and personal transformation, though it has received mixed reviews regarding its depth.

Testing VISUAL tool:
The posters listed in the search results primarily represent films that fall into the drama, action, and science fiction genres, with a focus on darker themes and moody atmospheres. 

1. **Dark Blue World** - This film, while a drama, likely has a somber tone given its title and the context of its narrative.
2. **Fifty Shades of Grey** - Although primarily a romance, it has elements of darkness and complexity in its themes, which may contribute to a moody aesthetic.
3. **The King** - This historical drama features darker themes of power and conflict

In [14]:
print("\n" + "=" * 60)
print("Testing COMBINED tool:")
print("=" * 60)
result = combined_tool.invoke("dark moody sci-fi films")
print(result)


Testing COMBINED tool:
For the category of "dark moody sci-fi films," the most relevant movie from the provided list is **"9" (2009)**. 

### Textual Alignment:
"9" is an animated film that fits the dark and moody sci-fi genre. It presents a post-apocalyptic world where sentient dolls, created from the remnants of humanity, struggle for survival against machines that have wiped out civilization. The film's atmosphere is filled with tension, isolation, and existential themes, similar to "Alien." Both films explore the fragility of life and the consequences of humanity's actions, making "9" a compelling choice for fans of dark sci-fi.

### Visual Alignment:
While the other films listed do not primarily fall into the sci-fi genre, "9" features a distinct visual style that complements its dark themes. The animation is gritty and atmospheric, enhancing the sense of dread and despair that permeates the story. This aligns with the haunting visuals mentioned in the description of "Alien."

##

#### Testing agent

In [4]:
from src.agents.movie_agent import MovieAgent

In [8]:
agent = MovieAgent(full_chain, visual_retriever)

queries = [
    "What is the Matrix about?",
    "Dark moody sci-fi genre films",
    "what themes does titanic touch?",
    "Movies with vibrant colors",
    "drama about love and lust with engaging red poster"
]

for query in queries:
    agent.query(query)
    print("\n")

Creating tools...
Building agent graph...
✓ Agent ready!
================================ Human Message =================================

What is the Matrix about?
================================== Ai Message ==================================
Tool Calls:
  search_movies_by_content (call_WxGQnsqm91rcG6SnAp0TC4fI)
 Call ID: call_WxGQnsqm91rcG6SnAp0TC4fI
  Args:
    query: What is The Matrix about?
================================= Tool Message =================================
Name: search_movies_by_content

The provided information does not include a description or details about "The Matrix." Therefore, I cannot answer your question about what "The Matrix" is about.
================================== Ai Message ==================================

"The Matrix" is a science fiction film that explores themes of reality, perception, and control. The story follows a computer hacker named Neo, who discovers that the world he lives in is a simulated reality created by sentient machines to s

In [11]:
_ = agent.query("recommend some scifi movie with yellow posters from the nineties")

================================ Human Message =================================

recommend some scifi movie with yellow posters from the nineties
================================== Ai Message ==================================
Tool Calls:
  search_movies_by_visual (call_QXKmGeqt0zmTXatOq2WG2qfc)
 Call ID: call_QXKmGeqt0zmTXatOq2WG2qfc
  Args:
    query: sci-fi movies with yellow posters from the nineties
================================= Tool Message =================================
Name: search_movies_by_visual

The posters listed in the search results share a common theme of being visually striking, with a focus on the science fiction genre, particularly in the context of the nineties. The films include a mix of action, adventure, horror, and drama, indicating a diverse range of storytelling within the sci-fi realm. 

1. **Virus (1999)** - This film combines elements of action, adventure, and horror, suggesting a thrilling narrative involving technology or extraterrestrial threats.

In [12]:
_ = agent.query("what's that movie where the kid is left alone")

================================ Human Message =================================

what's that movie where the kid is left alone
================================== Ai Message ==================================
Tool Calls:
  search_movies_by_content (call_gA5sadQAStQU6pRnR2RjmKNX)
 Call ID: call_gA5sadQAStQU6pRnR2RjmKNX
  Args:
    query: kid left alone
================================= Tool Message =================================
Name: search_movies_by_content

The movie "The Ant Bully" features a young boy named Lucas who is navigating his independence in a big world, which may resonate with the theme of a kid left alone. Additionally, "Walkabout" involves children who are left to survive in the wilderness, highlighting their struggle and need to return to civilization. Both films explore themes related to children facing challenges on their own.
================================== Ai Message ==================================

The movie you're thinking of might be "Home Alone," where